In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from logicpy import *
from logicpy.core import *
from logicpy.builtin import *
from logicpy.predicate import *
from logicpy.data import *
a = _

In [9]:
u, n = Universe().and_namespace()

n.bla[a.foo] = True
n.bla[a.bar] = True
n.bla[a.both(a.X, a.Y)] = n.bla(a.X) & n.bla(a.Y)

#[res for res, _ in zip(u.query(n.bla(a.Z)), range(3))]
u.simple_query(n.bla(a.Z))

[]

In [10]:
u, n = Universe().and_namespace()

n.test[a.foo(a.X), a.Y] = n.test(a.X, a.Y)
n.test[a.bar, a.bar] = True

e = n.test(a.foo(a.bar), a.Z)
#u.simple_query(e, debug=True)

[autoreload of logicpy.data failed: Traceback (most recent call last):
  File "/usr/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ValueError: __init__() requires a code object with 0 free vars, not 1
]


In [5]:
u, n = Universe().and_namespace()

n.sum[a.zero, a.X, a.X] = True
n.sum[a.next(a.X), a.Y, a.Z] = n.sum(a.X, a.next(a.Y), a.Z)

def peano(x):
    if x == 0: return a.zero
    return a.next(peano(x-1))

def inv_peano(x):
    if x == a.zero: return 0
    return 1+inv_peano(x.children[0])
    

inv_peano(u.simple_query(n.sum(peano(7), peano(5), a.X))[0]['X'])

12

In [5]:
u, n = Universe().and_namespace()
_ = Underscore()

(, _.Y == 8, _.Z << _.X + _.Y)

(False,
 False,
 Eval(Variable('Z'), Compound('+', (Variable('X'), Variable('Y')))))

In [4]:
u.interactive()

? quit
Not a provable structure: quit
? quit.


SyntaxError: unexpected EOF while parsing (<string>, line 1)

In [12]:
_ = Underscore()
c = _.foo(_.X, _.Y).with_scope(2)
d = _.foo(_.X, _.Y).with_scope(2)
c == d

True